In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import os

# Función Objetivo

In [2]:
def f(x,y):
    return 0.5 * (x**4 - 16 * x**2 + 5 * x) + 0.5 * (y**4 - 16 * y**2 + 5 * y)

# Minimos Locales

### (-2.90,2.74)  (2.74,-2.90) (2.74,2.74)

# Minimo Global

### (-2.90,-2.90)

Estos valores son aproximaciones de los minimos reales

# Gradiente de la función

In [5]:
def grad_f(x,y):
    df_dx = 2*x**3 - 16*x + 2.5
    df_dy = 2*y**3 - 16*y + 2.5
    return np.array([df_dx,df_dy])

# Inversa de la Hessiana de la función

In [6]:
def inv_hessian_f(x,y):
    hxx = 6*x**2 - 16
    hyy = 6*y**2 - 16
    if hxx == 0 or hyy == 0:
        raise ValueError("Hessiana no invertible en este punto.")
    return np.array([[1/hxx,0],[0,1/hyy]])

# Descenso de Gradiente

In [ ]:
def gradient_descent(f,grad_f,xo,yo,alpha = 0.001 , tol = 1e-6 , max_iter = 100):
    x = np.array([xo,yo],dtype=float)
    for i in range(max_iter):
        grad = grad_f(x[0],x[1])
        new_x = x - alpha * grad
        if abs(x[0]) > 1e6 or abs(x[1]) > 1e6:
            print("Divergencia detectada en la iteración", i)
            break
        
        if np.linalg.norm(new_x - x) < tol:
            print(f"Cambio pequeño en x en la iteración {i+1}")
            break
            
        x = new_x
    
    result = {  
        "method": "Descenso de Gradiente",
        "input": {"xo": xo, "yo": yo,"alpha":alpha,"tol": tol, "max_iter": max_iter},
        "output": {
            "x_final": float(x[0]),
            "y_final": float(x[1]),
            "f_final": float(f(x[0], x[1])),
            "iterations": i + 1
        }
    }

    filename = f"resultados_gradiente_.json"

    if os.path.exists(filename):
        with open(filename, "r") as f_in:
            data = json.load(f_in)
            
    else:
        data = []
    data.append(result)
    with open(filename, "w") as f_out:
        json.dump(data, f_out, indent=4)
    print(f"\n Resultados guardados en '{filename}'")

    return x,i,f(x[0],x[1])

# Método de Newton

In [ ]:
def newthon_method(f,grad_f,inv_hessian_f,xo,yo,tol = 1e-6,max_iter=100):
    x = np.array([xo,yo],dtype=float)
    for i in range(max_iter):
        grad = grad_f(x[0],x[1])
        H_inv = inv_hessian_f(x[0],x[1])

        step = H_inv @ grad
        new_x = x - step
        
        if np.linalg.norm(new_x - x) < tol:
            print(f"Cambio pequeño en x en la iteración {i+1}")
            break

        x = new_x

    result = {  
        "method": "Newton",
        "input": {"xo": xo, "yo": yo, "tol": tol, "max_iter": max_iter},
        "output": {
            "x_final": float(x[0]),
            "y_final": float(x[1]),
            "f_final": float(f(x[0], x[1])),
            "iterations": i + 1
        }
    }

    filename = f"resultados_newton_.json"

    if os.path.exists(filename):
        with open(filename, "r") as f_in:
            data = json.load(f_in)
    else:
        data = []
    data.append(result)
    with open(filename, "w") as f_out:
        json.dump(data, f_out, indent=4)
    print(f"\n Resultados guardados en '{filename}'")

    return x,i,f(x[0],x[1])

### Se tomó como criterio de parada el cambio pequeño de x de un paso hacia otro. La variable result se utiliza para guardar todos los datos de interes para su futura comparación

# Casos Prueba

Para correr los casos prueba debe rellenar las listas x_ini(x iniciales) , y_ini(y iniciales) y alphas(tamaños de saltos).Al correr los métodos de los algoritmos implementados se crean 2 archivos .json uno para cada método respectivamente . En dichos archivos se almacenarán todos los datos referentes a cada llamada realizada a los métodos.

In [ ]:
x_ini = [-2]
y_ini = [-2]
alphas = [0.001]
def tester(x_ini,y_ini,alphas):
    for i in range(len(x_ini)):
        newthon_method(f,grad_f,inv_hessian_f,x_ini[i],y_ini[i])
        gradient_descent(f,grad_f,x_ini[i],y_ini[i],alphas[i])

tester(x_ini,y_ini,alphas)

# Visualizar Salida
La funcionalidad de este apartado es analizar solamente el último llamado realizado a los métodos

In [ ]:
with open("resultados_newton_.json") as f_in1:
    data_newton  = json.load(f_in1)
    print(data_newton[-1])

with open("resultados_gradiente_.json") as f_in2:
    data_gradient = json.load(f_in2) 

print("Método:", data_newton[-1]['method'])
print("Iteraciones:", data_newton[-1]['output']['iterations'])
print("Valor de x:", data_newton[-1]['output']['x_final'])
print("Valor de y", data_newton[-1]['output']['y_final'])
print("Valor final:",data_newton[-1]['output']['f_final'])
print("=================================================================")
print("Método:", data_gradient[-1]['method'])
print("Iteraciones:", data_gradient[-1]['output']['iterations'])
print("Valor de x:", data_gradient[-1]['output']['x_final'])
print("Valor de y:",data_gradient[-1]['output']['y_final'])
print("Valor final:",data_gradient[-1]['output']['f_final'])

# Tabla de Comparación

Aquí se mostrara una comparación entre los métodos en cada una de las llamadas realizadas

In [ ]:
# --- Cargar los JSON ---
with open("resultados_newton_.json", "r") as f:
    newton_data = json.load(f)

with open("resultados_gradiente_.json", "r") as f:
    grad_data = json.load(f)

# --- Crear tabla comparativa ---
tabla = []

for n, g in zip(newton_data, grad_data):
    fila = {
        # Columnas Newton
        'X inicial Newton': n['input']['xo'],
        'Y inicial Newton': n['input']['yo'],
        'X final Newton': n['output']['x_final'],
        'Y final Newton': n['output']['y_final'],
        'Valor final Newton': n['output']['f_final'],
        'Iteraciones Newton': n['output']['iterations'],

        # Columnas Descenso de Gradiente
        'Tamaño de paso': g['input']['alpha'],
        'X inicial Gradiente': g['input']['xo'],
        'Y inicial Gradiente': g['input']['yo'],
        'X final Gradiente': g['output']['x_final'],
        'Y final Gradiente': g['output']['y_final'],
        'Valor final Gradiente': g['output']['f_final'],
        'Iteraciones Gradiente': g['output']['iterations'],
    }
    tabla.append(fila)

# --- Crear DataFrame ---
df_comparativa = pd.DataFrame(tabla)

# --- Mostrar tabla completa ---
# Mostrar todas las columnas y expandir ancho máximo
pd.set_option("display.max_columns", None)  # mostrar todas las columnas
pd.set_option("display.width", 200)         # ancho total de la consola
pd.set_option("display.expand_frame_repr", False)
print(df_comparativa)
